Chest X-ray (Pneumonia): Classification Model Using Transfer Learning

LIST OF IMPORTS

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Dropout,GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix

DATA PREPARATION

In [2]:
# The paths to the train, validation, and test directories
train_path = "chest_xray/train"
val_path = "chest_xray/val"
test_path = "chest_xray/test"

# The input image size
input_size = (224, 224)

# The batch size for the data generators
batch_size = 32

In [3]:
# The data generators for the training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=input_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_path, target_size=input_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=input_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Pre_Trained Model RESNET50

In [5]:
# The pre-trained model to use as a feature extractor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [6]:
# The number of classes in the dataset
num_classes = 2

In [13]:
# Add a global average pooling layer and a fully connected layer on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)


In [14]:
# The model
model_RES = Model(inputs=base_model.input, outputs=predictions)

In [15]:
model_RES.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [16]:
# Compile the model
model_RES.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Train the model
model_RES.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=val_generator, validation_steps=len(val_generator))

Epoch 1/10
163/163 [==============================] - 225s 1s/step - loss: 0.5569 - accuracy: 0.7366 - val_loss: 0.6244 - val_accuracy: 0.6250
Epoch 2/10
163/163 [==============================] - 235s 1s/step - loss: 0.4197 - accuracy: 0.7993 - val_loss: 0.8494 - val_accuracy: 0.5625
Epoch 3/10
163/163 [==============================] - 222s 1s/step - loss: 0.4111 - accuracy: 0.8125 - val_loss: 0.8069 - val_accuracy: 0.6250
Epoch 4/10
163/163 [==============================] - 221s 1s/step - loss: 0.3743 - accuracy: 0.8284 - val_loss: 0.6263 - val_accuracy: 0.6875
Epoch 5/10
163/163 [==============================] - 219s 1s/step - loss: 0.3461 - accuracy: 0.8436 - val_loss: 0.9934 - val_accuracy: 0.6250
Epoch 6/10
163/163 [==============================] - 221s 1s/step - loss: 0.3467 - accuracy: 0.8441 - val_loss: 1.0766 - val_accuracy: 0.6250
Epoch 7/10
163/163 [==============================] - 221s 1s/step - loss: 0.3321 - accuracy: 0.8549 - val_loss: 1.2925 - val_accuracy: 0.5625

In [18]:
# Evaluate the model on the test set
test_loss_RES, test_acc_RES = model_RES.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', test_loss_RES)
print('Test accuracy:', test_acc_RES)

20/20 [==============================] - 25s 1s/step - loss: 0.3848 - accuracy: 0.8446
Test loss: 0.3847949206829071
Test accuracy: 0.8445512652397156


In [19]:
# Make predictions on the test set
test_generator.reset()
preds = model_RES.predict(test_generator, steps=len(test_generator), verbose=1)
y_true = test_generator.classes
y_pred = np.argmax(preds, axis=1)

20/20 [==============================] - 26s 1s/step


In [20]:
#Print the classification report and confusion matrix
print('Classification Report:')
print(classification_report(y_true, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.83      0.80       234
           1       0.89      0.85      0.87       390

    accuracy                           0.84       624
   macro avg       0.83      0.84      0.84       624
weighted avg       0.85      0.84      0.85       624

Confusion Matrix:
[[195  39]
 [ 58 332]]


USING VGG

In [31]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential

In [17]:
# Load a pre-trained VGG16 model and freeze its layers
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))
vgg16.trainable = False

58889256/58889256 [==============================] - 54s 1us/step


In [22]:
# Create a new model that includes the VGG16 layers and adds a few new layers on top
model_VGG = Sequential([
    vgg16,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')
])

In [28]:
model_VGG.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 27,561,282
Trainable params: 12,846,594
Non-trainable params: 14,714,688
_________________________________________________________________


In [23]:
# Compile the model
model_VGG.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Train the model
model_VGG.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=val_generator, validation_steps=len(val_generator))

Epoch 1/10
163/163 [==============================] - 438s 3s/step - loss: 0.3940 - accuracy: 0.9047 - val_loss: 0.3413 - val_accuracy: 0.8750
Epoch 2/10
163/163 [==============================] - 439s 3s/step - loss: 0.1480 - accuracy: 0.9429 - val_loss: 0.6323 - val_accuracy: 0.6875
Epoch 3/10
163/163 [==============================] - 439s 3s/step - loss: 0.1355 - accuracy: 0.9480 - val_loss: 0.6952 - val_accuracy: 0.6875
Epoch 4/10
163/163 [==============================] - 438s 3s/step - loss: 0.1297 - accuracy: 0.9526 - val_loss: 0.2359 - val_accuracy: 0.8750
Epoch 5/10
163/163 [==============================] - 438s 3s/step - loss: 0.1417 - accuracy: 0.9528 - val_loss: 0.3707 - val_accuracy: 0.8125
Epoch 6/10
163/163 [==============================] - 440s 3s/step - loss: 0.1313 - accuracy: 0.9503 - val_loss: 0.4571 - val_accuracy: 0.7500
Epoch 7/10
163/163 [==============================] - 438s 3s/step - loss: 0.1253 - accuracy: 0.9525 - val_loss: 0.5488 - val_accuracy: 0.6875

In [25]:
# Evaluate the model on the test set
test_loss_VGG, test_acc_VGG = model_VGG.evaluate(test_generator, steps=len(test_generator))
print('Test loss:', test_loss_VGG)
print('Test accuracy:', test_acc_VGG)

20/20 [==============================] - 51s 3s/step - loss: 0.2070 - accuracy: 0.9311
Test loss: 0.20698951184749603
Test accuracy: 0.9310897588729858


In [26]:
# Make predictions on the test set
test_generator.reset()
preds = model_VGG.predict(test_generator, steps=len(test_generator), verbose=1)
y_true = test_generator.classes
y_pred = np.argmax(preds, axis=1)

20/20 [==============================] - 51s 3s/step


In [27]:
#Print the classification report and confusion matrix
print('Classification Report:')
print(classification_report(y_true, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.87      0.90       234
           1       0.93      0.97      0.95       390

    accuracy                           0.93       624
   macro avg       0.93      0.92      0.93       624
weighted avg       0.93      0.93      0.93       624

Confusion Matrix:
[[204  30]
 [ 13 377]]
